In [ ]:
# Boilerplate that all notebooks reuse:
from analysis_common import *

%matplotlib inline

# Kernel analysis

In [ ]:
df = read_ods("./results.ods", "matmul-kernel")

expand_modes(df)

print(df["MODE"].unique())
#############################################
# Disregard the store result for the kernel #
#############################################
df.loc[df["MODE"] == "AD (volatile result)", "MODE"] = "AD"
order = ['DRAM', 'AD', 'AD (in-place FMA)', 'MM (hot)', 'MM (cold)']
hue_order = [7000, 1000]
# Split the two families of experiments
df_rowcol = df[df.MATRIX_SIDE != 0]
df = df[df.MATRIX_SIDE == 0]

sns.barplot(x='MODE', y='TIMING',
            data=df[(df.BLOCKSIZE == 1000)],
            capsize=0.1,
            order=order,
            palette=custom_kernel_palette(6))
plt.title("Submatrix size: 1000x1000 (small object)")
plt.xticks(rotation=25, horizontalalignment='right')
plt.show()

sns.barplot(x='MODE', y='TIMING',
            data=df[(df.BLOCKSIZE == 7000)],
            capsize=0.1,
            order=order,
            palette=custom_kernel_palette(6))
plt.title("Submatrix size: 7000x7000 (big object)")
plt.xticks(rotation=25, horizontalalignment='right')
plt.show()

###################################

# sns.barplot(x='MODE', y='TIMING',
#             data=df_rowcol[(df_rowcol.BLOCKSIZE == 1000)],
#             capsize=0.1,
#             order=order,
#             palette=palette)
# plt.title("BLOCKSIZE: 1k || row x col")
# plt.xticks(rotation=25, horizontalalignment='right')
# plt.show()

# sns.barplot(x='MODE', y='TIMING',
#             data=df_rowcol[(df_rowcol.BLOCKSIZE == 7000)],
#             capsize=0.1,
#             order=order,
#             palette=palette)
# plt.title("BLOCKSIZE: 7k || row x col")
# plt.xticks(rotation=25, horizontalalignment='right')
# plt.show()

# Remove MM-NVM as it is outlier-ish
#df = df[df.MODE != 'MM-NVM']
# ... or maybe not? trying set_ylim maybe:
#axes = plt.gca()
#axes.set_ylim([0,1.5])
#plt.title("...")
#plt.show()

In [ ]:
df.loc[(df.BLOCKSIZE == 1000), "NORMALIZED"] = df.TIMING 
df.loc[(df.BLOCKSIZE == 7000), "NORMALIZED"] = df.TIMING / (7*7*7)

ax = sns.barplot(y='MODE', x='NORMALIZED',
                 data=df,
                 capsize=0.1,
                 order=order,
                 hue_order=hue_order,
                 hue="BLOCKSIZE",
                 palette="muted")

kernel_plot_tweaks(ax, 7*7*7, legend_title="Submatrix blocksize")

plt.savefig("matmul-kernel.pdf", bbox_inches='tight')
plt.show()


In [ ]:
kernel_times = df.groupby(["BLOCKSIZE", "MODE"]).min()
kernel_times

In [ ]:
#rowcol_times = df_rowcol.groupby(["BLOCKSIZE", "MODE"]).min()
#rowcol_times

# Matmul results analysis

In [ ]:
df = read_ods("./results.ods", "matmul-app")
expand_modes(df)
df

In [ ]:
for bs in [1000, 7000]:
    df.loc[(df.BLOCKSIZE == bs) & (df.MODE == "DRAM"), "ATOM_KERNEL"] =  \
        kernel_times.loc[(bs, "DRAM"), "TIMING"]
    df.loc[(df.BLOCKSIZE == bs) & (df.MODE == "AD (volatile result)"), "ATOM_KERNEL"] =  \
        kernel_times.loc[(bs, "AD"), "TIMING"]
    df.loc[(df.BLOCKSIZE == bs) & (df.MODE == "AD (store result)"), "ATOM_KERNEL"] =  \
        kernel_times.loc[(bs, "AD"), "TIMING"]
    df.loc[(df.BLOCKSIZE == bs) & (df.MODE == "AD (in-place FMA)"), "ATOM_KERNEL"] =  \
        kernel_times.loc[(bs, "AD (in-place FMA)"), "TIMING"]
    df.loc[(df.BLOCKSIZE == bs) & (df.MODE == "DAOS (volatile result)"), "ATOM_KERNEL"] =  \
        kernel_times.loc[(bs, "DRAM"), "TIMING"]
    df.loc[(df.BLOCKSIZE == bs) & (df.MODE == "DAOS (store result)"), "ATOM_KERNEL"] =  \
        kernel_times.loc[(bs, "DRAM"), "TIMING"]

df.loc[(df.BLOCKSIZE == 1000) 
       & (df.MATRIX_SIDE == 42) 
       & (df.MODE == "MM"),
       "ATOM_KERNEL"] = kernel_times.loc[(1000, "MM (hot)"), "TIMING"]
df.loc[(df.BLOCKSIZE == 7000) 
       & (df.MATRIX_SIDE == 6)
       & (df.MODE == "MM"),
       "ATOM_KERNEL"] = kernel_times.loc[(7000, "MM (hot)"), "TIMING"]
df.loc[(df.BLOCKSIZE == 1000) 
       & (df.MATRIX_SIDE == 84)
       & (df.MODE == "MM"),
       "ATOM_KERNEL"] = kernel_times.loc[(1000, "MM (cold)"), "TIMING"]
df.loc[(df.BLOCKSIZE == 7000) 
       & (df.MATRIX_SIDE == 12)
       & (df.MODE == "MM"),
       "ATOM_KERNEL"] = kernel_times.loc[(7000, "MM (cold)"), "TIMING"]

df["KERNEL_TIME"] = df["MATRIX_SIDE"]**3 * df["ATOM_KERNEL"]

# Sanity check
null_values = df[df.isnull().values]
if len(null_values) > 0:
    print('There are null values, check null_values variable')

In [ ]:
df

# Article image generation

In [ ]:
sns.set(style="whitegrid")

order = ['DRAM', 'AD (volatile result)', 'AD (store result)', 'AD (in-place FMA)', 
         'MM', 'DAOS (volatile result)', 'DAOS (store result)']

small = (
    ((df.BLOCKSIZE == 1000) & (df.MATRIX_SIDE == 42)) |
    ((df.BLOCKSIZE == 7000) & (df.MATRIX_SIDE == 6))
)

big = (
    ((df.BLOCKSIZE == 1000) & (df.MATRIX_SIDE == 84)) |
    ((df.BLOCKSIZE == 7000) & (df.MATRIX_SIDE == 12))
)

ax = sns.barplot(y='MODE', x="TIMING",
                 data=df[small],
                 capsize=0.1,
                 order=order,
                 hue_order=hue_order,
                 palette="colorblind",
                 hue=df.BLOCKSIZE)

bottom = sns.barplot(y='MODE', x="KERNEL_TIME",
            data=df[small],
            capsize=0,
            order=order,
            hue_order=hue_order,
            palette="pastel",
            hue=df.BLOCKSIZE)

crop_axis(ax, 800)
ylabel_tweaks(ax, [2, 5], ['non-active', 'active'], 0.40, 0.005)
legend_tweaks(bottom, ["big objects", "small objects", "kernel comp."], placement='upper center')
ax.set_xlabel("execution time (s)")
plt.title("Small dataset")
save_tweaks("matmul-small.pdf", big=True)
plt.show()

ax = sns.barplot(y='MODE', x="TIMING",
                 data=df[big],
                 capsize=0.1,
                 order=order,
                 hue_order=hue_order,
                 palette="colorblind",
                 hue=df.BLOCKSIZE)

annotate_dram(ax)

bottom = sns.barplot(y='MODE', x="KERNEL_TIME",
            data=df[big],
            capsize=0,
            order=order,
            hue_order=hue_order,
            palette="pastel",
            hue=df.BLOCKSIZE)

crop_axis(ax, 6000)
ylabel_tweaks(ax, [2, 5], ['non-active', 'active'], 0.40, 0.005)
legend_tweaks(bottom, ["big objects", "small objects", "kernel comp."], placement='upper center')
ax.set_xlabel("execution time (s)")
plt.title("Big dataset")
save_tweaks("matmul-big.pdf", big=True)
plt.show()

In [ ]:
df.groupby(["BLOCKSIZE", "MATRIX_SIDE", "MODE"]).mean()